In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from pandarallel import pandarallel


In [3]:
import nltk
corp = nltk.corpus.ConllCorpusReader('.', 'tiger_release_aug07.corrected.16012013.conll09',
                                     ['ignore', 'words', 'ignore', 'ignore', 'pos'],
                                     encoding='utf-8')

In [4]:
# displaying options
pd.set_option('display.float_format', '{:.20f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# deprecation warning set off
pd.options.mode.chained_assignment = None 

In [22]:
data = pd.read_csv('full_analysis.csv')

In [80]:
zehring = pd.read_csv('Querdenken_Labeled_Backbone.csv')

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt

import nltk
corp = nltk.corpus.ConllCorpusReader('.', 'tiger_release_aug07.corrected.16012013.conll09',
                                     ['ignore', 'words', 'ignore', 'ignore', 'pos'],
                                     encoding='utf-8')

import random

tagged_sents = list(corp.tagged_sents())
random.shuffle(tagged_sents)

# set a split size: use 90% for training, 10% for testing
split_perc = 0.1
split_size = int(len(tagged_sents) * split_perc)
train_sents, test_sents = tagged_sents[split_size:], tagged_sents[:split_size]

from ClassifierBasedGermanTagger.ClassifierBasedGermanTagger import ClassifierBasedGermanTagger

tagger = ClassifierBasedGermanTagger(train=train_sents)
accuracy = tagger.evaluate(test_sents)
accuracy

0.9403395692501179

In [7]:
def action_quotient(text): 
    
    verbs = ["VAFIN", "VVFIN", "VMFIN", "VVPP", "VVINF", "VAINF"]
    adjectives = ["ADJD"]
    
    split = text.split()
    
    if len(split) > 5000:
        split = random.sample(split, 5000)
    
    bundle = tagger.tag(split)
    
    v = [t for t in bundle if t[1] in verbs]
    a = [t for t in bundle if t[1] in adjectives]
    
    quotient = len(a) and len(v)/len(a) or 0
    
    return quotient

In [35]:
with open('final_key_dict.pkl', 'rb') as fp:
    final_key_dict = pickle.load(fp)

In [ ]:
data['action_quotient'] = data['messages_processed'].parallel_apply(action_quotient)

In [36]:
pandarallel.initialize(nb_workers = 50, progress_bar = True)


INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [32]:
data.to_csv('fa_POS.csv.gzip', compression = 'gzip')

In [4]:
import random

tagged_sents = list(corp.tagged_sents())
random.shuffle(tagged_sents)

# set a split size: use 90% for training, 10% for testing
split_perc = 0.1
split_size = int(len(tagged_sents) * split_perc)
train_sents, test_sents = tagged_sents[split_size:], tagged_sents[:split_size]

In [8]:
from ClassifierBasedGermanTagger.ClassifierBasedGermanTagger import ClassifierBasedGermanTagger


In [9]:
tagger = ClassifierBasedGermanTagger(train=train_sents)

In [10]:
accuracy = tagger.evaluate(test_sents)

In [11]:
accuracy

0.9402547524628723

In [16]:
df = pd.DataFrame(bundle, columns = ['word', 'tag'])

In [20]:
df.tag.unique()

array(['CARD', 'VAFIN', 'ART', 'ADJA', 'APPR', 'ADJD', 'KON', 'PTKNEG',
       'PPER', 'VVFIN', 'PIS', 'KOKOM', 'VMFIN', 'ADV', '$,', 'PWAV',
       'VVPP', 'VVINF', 'PIAT', 'PWS', 'PTKA', 'KOUS', 'PDS', 'APPRART',
       'NN', 'PRF', 'VMPP', 'PTKZU', 'VAINF', 'VVIZU', 'PPOSAT', 'PROAV',
       'VMINF', 'VAIMP', 'PTKVZ', 'FM', 'APZR', '$(', 'NE', 'PWAT'],
      dtype=object)

In [1]:
def action_quotient(text):
    
    verbs = ["VAFIN", "VVFIN", "VMFIN", "VVPP", "VVINF", "VAINF"]
    adjectives = ["ADJD"]
    
    bundle = tagger.tag(text.split())
    
    v = [t for t in bundle if t[1] in verbs]
    a = [t for t in bundle if t[1] in adjectives]
    
    quotient = len(v)/len(a)
    
    return quotient

In [29]:
len(df.query('tag in @verbs'))/len(df.query('tag in @adjectives'))

2.586206896551724

In [63]:
with open('group_name_dictionary.pkl', 'rb') as fp:
    group_names_dict = pickle.load(fp)

In [70]:
group_names = [key.lower() for key, value in group_names_dict.items()]

In [243]:
def clean_names(name_list):
    new_name = []
    for i in name_list:
        i = i.replace(" ", "")
        name = ''
        for c in i:
            #if (c>= 'a' and c<='z') or (c>= 'A' and c<='Z') or (c>= '0' and c<='9') or (c == "_") or (c in ['ä', 'ö', 'ü']):
            if c.isalpha() or c.isnumeric():
                name += c.replace('ü', 'ue').replace('ö', 'oe').replace('ä', 'ae')#.replace(c,c)

        new_name.append(name)  
        
    return new_name

In [244]:
new_name = clean_names(group_names)

In [249]:
new_zehring = clean_names(zehring.channel_lower.tolist())

In [251]:
intersection = [name for name in new_name if name in new_zehring]

In [255]:
len(intersection)/len(new_zehring)

0.2585278276481149

In [254]:
len(new_zehring)

557

In [217]:
name = ''
for i in group_names[4]:
    if i.isnumeric() or i.isalpha():
        name += i
print(name)  

miteinanderseinplz1
